In [35]:
import requests
import config
import mysql.connector
import json
from bs4 import BeautifulSoup
import time

In [7]:
def get_grossing_stars(a_url):
    html_page = requests.get(a_url)
    soup = BeautifulSoup(html_page.content, "html.parser")
    div = soup.find("tbody")
    grs_stars = list(map(lambda x: x.text, div.findAll('td')))
    return grs_stars

In [24]:
def retrieve_top_grossing_stars (grossing_stars):
    top_g_s = []
    i = 0
    for num in range(int( len(grossing_stars) / 5 )):
        star = dict([('rank', int(grossing_stars[i])), ('name', grossing_stars[i+1]), 
                       ('WW_box', float(grossing_stars[i+2].replace(",",'').replace('$',''))),
                      ('num_movies', int(grossing_stars[i+3])),
                       ('avg_per_movie', float(grossing_stars[i+4].replace(",",'').replace('$','')))])
        i += 5
        top_g_s.append(star)
    return top_g_s

In [25]:
top_grossing_stars = []

# retrieve data for first 100
url = "https://www.the-numbers.com/box-office-star-records/worldwide/lifetime-acting/top-grossing-leading-stars"
grossing_stars = get_grossing_stars(url)
top_grossing_stars = retrieve_top_grossing_stars(grossing_stars)

# retrieve data for 100+
for i in range(101, 601, 100):
    url = "https://www.the-numbers.com/box-office-star-records/worldwide/lifetime-acting/top-grossing-leading-stars/{}".format(i)
    grossing_stars = get_grossing_stars(url)
    top_grossing_stars.extend (retrieve_top_grossing_stars(grossing_stars))    

In [18]:
# top_grossing_stars


In [17]:
len(top_grossing_stars)

600

In [32]:
conn = mysql.connector.connect(
                host = config.host,
                user = config.user,
                passwd = config.password,
                database = 'movie_insights'
                )
c = conn.cursor()

In [26]:
top_grossing_stars

[{'rank': 1,
  'name': 'Robert Downey, Jr.',
  'WW_box': 14135538425.0,
  'num_movies': 43,
  'avg_per_movie': 328733452.0},
 {'rank': 2,
  'name': 'Samuel L. Jackson',
  'WW_box': 14125127457.0,
  'num_movies': 61,
  'avg_per_movie': 231559467.0},
 {'rank': 3,
  'name': 'Scarlett Johansson',
  'WW_box': 13574501487.0,
  'num_movies': 29,
  'avg_per_movie': 468086258.0},
 {'rank': 4,
  'name': 'Chris Hemsworth',
  'WW_box': 11193459657.0,
  'num_movies': 22,
  'avg_per_movie': 508793621.0},
 {'rank': 5,
  'name': 'Zoe Saldana',
  'WW_box': 10903702398.0,
  'num_movies': 26,
  'avg_per_movie': 419373169.0},
 {'rank': 6,
  'name': 'Chris Evans',
  'WW_box': 10739097924.0,
  'num_movies': 25,
  'avg_per_movie': 429563917.0},
 {'rank': 7,
  'name': 'Chris Pratt',
  'WW_box': 10539482765.0,
  'num_movies': 17,
  'avg_per_movie': 619969574.0},
 {'rank': 8,
  'name': 'Tom Hanks',
  'WW_box': 10422066150.0,
  'num_movies': 51,
  'avg_per_movie': 204354238.0},
 {'rank': 9,
  'name': 'Bradley Co

In [28]:
def populate_db(records):
    INSERT_STR = f'INSERT INTO top_grossing_stars (rank, name, WW_box, num_movies, avg_per_movie) ' 
    VALUE_STR = 'VALUES (%s, %s, %s, %s, %s)'
    INSERT_QUERY = INSERT_STR + VALUE_STR
    for rec in records:
        values = tuple(rec.values())
        c.execute(INSERT_QUERY, values)
        conn.commit()
        print(rec)
    c.close()
    conn.close()    


In [29]:
populate_db(top_grossing_stars)

{'rank': 1, 'name': 'Robert Downey, Jr.', 'WW_box': 14135538425.0, 'num_movies': 43, 'avg_per_movie': 328733452.0}
{'rank': 2, 'name': 'Samuel L. Jackson', 'WW_box': 14125127457.0, 'num_movies': 61, 'avg_per_movie': 231559467.0}
{'rank': 3, 'name': 'Scarlett Johansson', 'WW_box': 13574501487.0, 'num_movies': 29, 'avg_per_movie': 468086258.0}
{'rank': 4, 'name': 'Chris Hemsworth', 'WW_box': 11193459657.0, 'num_movies': 22, 'avg_per_movie': 508793621.0}
{'rank': 5, 'name': 'Zoe Saldana', 'WW_box': 10903702398.0, 'num_movies': 26, 'avg_per_movie': 419373169.0}
{'rank': 6, 'name': 'Chris Evans', 'WW_box': 10739097924.0, 'num_movies': 25, 'avg_per_movie': 429563917.0}
{'rank': 7, 'name': 'Chris Pratt', 'WW_box': 10539482765.0, 'num_movies': 17, 'avg_per_movie': 619969574.0}
{'rank': 8, 'name': 'Tom Hanks', 'WW_box': 10422066150.0, 'num_movies': 51, 'avg_per_movie': 204354238.0}
{'rank': 9, 'name': 'Bradley Cooper', 'WW_box': 10386239368.0, 'num_movies': 26, 'avg_per_movie': 399470745.0}
{'r

{'rank': 76, 'name': 'Mel Gibson', 'WW_box': 4299689362.0, 'num_movies': 44, 'avg_per_movie': 97720213.0}
{'rank': 77, 'name': 'Ralph Fiennes', 'WW_box': 4296884374.0, 'num_movies': 20, 'avg_per_movie': 214844219.0}
{'rank': 78, 'name': 'Arnold Schwarzenegger', 'WW_box': 4216772686.0, 'num_movies': 32, 'avg_per_movie': 131774146.0}
{'rank': 79, 'name': 'Denzel Washington', 'WW_box': 4206332773.0, 'num_movies': 46, 'avg_per_movie': 91442017.0}
{'rank': 80, 'name': 'Jack Black', 'WW_box': 4202020805.0, 'num_movies': 23, 'avg_per_movie': 182696557.0}
{'rank': 81, 'name': 'Keira Knightley', 'WW_box': 4150757670.0, 'num_movies': 25, 'avg_per_movie': 166030307.0}
{'rank': 82, 'name': 'Idris Elba', 'WW_box': 4123735590.0, 'num_movies': 24, 'avg_per_movie': 171822316.0}
{'rank': 83, 'name': 'Geoffrey Rush', 'WW_box': 4096705600.0, 'num_movies': 14, 'avg_per_movie': 292621829.0}
{'rank': 84, 'name': 'Mike Myers', 'WW_box': 4086054579.0, 'num_movies': 13, 'avg_per_movie': 314311891.0}
{'rank': 8

{'rank': 152, 'name': "Lupita Nyong'o", 'WW_box': 2575405546.0, 'num_movies': 6, 'avg_per_movie': 429234258.0}
{'rank': 153, 'name': 'Rachel McAdams', 'WW_box': 2570586744.0, 'num_movies': 21, 'avg_per_movie': 122408893.0}
{'rank': 154, 'name': 'Antonio Banderas', 'WW_box': 2565265622.0, 'num_movies': 22, 'avg_per_movie': 116602983.0}
{'rank': 155, 'name': 'Matthew McConaughey', 'WW_box': 2552998800.0, 'num_movies': 26, 'avg_per_movie': 98192262.0}
{'rank': 156, 'name': 'Zendaya', 'WW_box': 2542661222.0, 'num_movies': 5, 'avg_per_movie': 508532244.0}
{'rank': 157, 'name': 'Jake Gyllenhaal', 'WW_box': 2529429797.0, 'num_movies': 23, 'avg_per_movie': 109975209.0}
{'rank': 158, 'name': 'Angela Bassett', 'WW_box': 2524590159.0, 'num_movies': 8, 'avg_per_movie': 315573770.0}
{'rank': 159, 'name': 'Emma Thompson', 'WW_box': 2491245404.0, 'num_movies': 15, 'avg_per_movie': 166083027.0}
{'rank': 160, 'name': 'Zach Galifianakis', 'WW_box': 2481086750.0, 'num_movies': 12, 'avg_per_movie': 206757

{'rank': 228, 'name': 'Sarah Vowell', 'WW_box': 1857242740.0, 'num_movies': 2, 'avg_per_movie': 928621370.0}
{'rank': 229, 'name': 'Bill Nighy', 'WW_box': 1854043816.0, 'num_movies': 16, 'avg_per_movie': 115877739.0}
{'rank': 230, 'name': 'Emma Stone', 'WW_box': 1854041488.0, 'num_movies': 15, 'avg_per_movie': 123602766.0}
{'rank': 231, 'name': 'Benicio del Toro', 'WW_box': 1848215768.0, 'num_movies': 14, 'avg_per_movie': 132015412.0}
{'rank': 232, 'name': 'Megan Fox', 'WW_box': 1847448183.0, 'num_movies': 9, 'avg_per_movie': 205272020.0}
{'rank': 233, 'name': 'Drew Barrymore', 'WW_box': 1847028958.0, 'num_movies': 30, 'avg_per_movie': 61567632.0}
{'rank': 234, 'name': 'T.J. Miller', 'WW_box': 1845323397.0, 'num_movies': 8, 'avg_per_movie': 230665425.0}
{'rank': 235, 'name': 'Luke Evans', 'WW_box': 1845285721.0, 'num_movies': 11, 'avg_per_movie': 167753247.0}
{'rank': 236, 'name': 'RenÃ©e Zellweger', 'WW_box': 1834338189.0, 'num_movies': 23, 'avg_per_movie': 79753834.0}
{'rank': 237, '

{'rank': 304, 'name': 'John Malkovich', 'WW_box': 1410549023.0, 'num_movies': 24, 'avg_per_movie': 58772876.0}
{'rank': 305, 'name': 'Jordana Brewster', 'WW_box': 1409394782.0, 'num_movies': 6, 'avg_per_movie': 234899130.0}
{'rank': 306, 'name': 'Marisa Tomei', 'WW_box': 1405164441.0, 'num_movies': 14, 'avg_per_movie': 100368889.0}
{'rank': 307, 'name': 'Matthew Gray Gubler', 'WW_box': 1404715280.0, 'num_movies': 6, 'avg_per_movie': 234119213.0}
{'rank': 308, 'name': 'Julianne Moore', 'WW_box': 1404632514.0, 'num_movies': 31, 'avg_per_movie': 45310726.0}
{'rank': 309, 'name': 'Tom Wilkinson', 'WW_box': 1404432063.0, 'num_movies': 11, 'avg_per_movie': 127675642.0}
{'rank': 310, 'name': 'Michelle Monaghan', 'WW_box': 1399004908.0, 'num_movies': 14, 'avg_per_movie': 99928922.0}
{'rank': 311, 'name': 'Donald Glover', 'WW_box': 1393165400.0, 'num_movies': 4, 'avg_per_movie': 348291350.0}
{'rank': 312, 'name': 'Wesley Snipes', 'WW_box': 1383572191.0, 'num_movies': 27, 'avg_per_movie': 512434

{'rank': 380, 'name': 'Sofia Boutella', 'WW_box': 1164945445.0, 'num_movies': 5, 'avg_per_movie': 232989089.0}
{'rank': 381, 'name': 'Danny Glover', 'WW_box': 1161121270.0, 'num_movies': 18, 'avg_per_movie': 64506737.0}
{'rank': 382, 'name': 'Naomi Scott', 'WW_box': 1153686049.0, 'num_movies': 3, 'avg_per_movie': 384562016.0}
{'rank': 383, 'name': 'Shailene Woodley', 'WW_box': 1151092677.0, 'num_movies': 11, 'avg_per_movie': 104644789.0}
{'rank': 384, 'name': 'Yahya Abdul-Mateen II', 'WW_box': 1146894640.0, 'num_movies': 2, 'avg_per_movie': 573447320.0}
{'rank': 385, 'name': 'Eric Bana', 'WW_box': 1142296144.0, 'num_movies': 14, 'avg_per_movie': 81592582.0}
{'rank': 386, 'name': 'William Moseley', 'WW_box': 1137880860.0, 'num_movies': 2, 'avg_per_movie': 568940430.0}
{'rank': 387, 'name': 'Anna Popplewell', 'WW_box': 1137880860.0, 'num_movies': 3, 'avg_per_movie': 379293620.0}
{'rank': 388, 'name': 'Stellan SkarsgÃ¥rd', 'WW_box': 1137480040.0, 'num_movies': 14, 'avg_per_movie': 8124857

{'rank': 456, 'name': 'Giancarlo Esposito', 'WW_box': 985956731.0, 'num_movies': 6, 'avg_per_movie': 164326122.0}
{'rank': 457, 'name': 'Mark Strong', 'WW_box': 981718778.0, 'num_movies': 12, 'avg_per_movie': 81809898.0}
{'rank': 458, 'name': 'Kaya Scodelario', 'WW_box': 980954019.0, 'num_movies': 7, 'avg_per_movie': 140136288.0}
{'rank': 459, 'name': 'Aaron Eckhart', 'WW_box': 976530094.0, 'num_movies': 20, 'avg_per_movie': 48826505.0}
{'rank': 460, 'name': 'Christina Ricci', 'WW_box': 975171715.0, 'num_movies': 17, 'avg_per_movie': 57363042.0}
{'rank': 461, 'name': 'Justin Timberlake', 'WW_box': 968975976.0, 'num_movies': 9, 'avg_per_movie': 107663997.0}
{'rank': 462, 'name': 'Justin Bartha', 'WW_box': 967494010.0, 'num_movies': 6, 'avg_per_movie': 161249002.0}
{'rank': 463, 'name': 'Conrad Vernon', 'WW_box': 966623305.0, 'num_movies': 2, 'avg_per_movie': 483311653.0}
{'rank': 464, 'name': 'Chris Miller', 'WW_box': 966623305.0, 'num_movies': 2, 'avg_per_movie': 483311653.0}
{'rank': 

{'rank': 534, 'name': 'Scott Eastwood', 'WW_box': 809862815.0, 'num_movies': 7, 'avg_per_movie': 115694688.0}
{'rank': 535, 'name': 'Jack McBrayer', 'WW_box': 807811005.0, 'num_movies': 5, 'avg_per_movie': 161562201.0}
{'rank': 536, 'name': 'Hilary Swank', 'WW_box': 806117857.0, 'num_movies': 15, 'avg_per_movie': 53741190.0}
{'rank': 537, 'name': 'Daniel Day-Lewis', 'WW_box': 805750555.0, 'num_movies': 15, 'avg_per_movie': 53716704.0}
{'rank': 538, 'name': 'Ethan Hawke', 'WW_box': 804183024.0, 'num_movies': 35, 'avg_per_movie': 22976658.0}
{'rank': 539, 'name': 'Mark Rylance', 'WW_box': 800930501.0, 'num_movies': 6, 'avg_per_movie': 133488417.0}
{'rank': 540, 'name': 'Terrence Howard', 'WW_box': 798395513.0, 'num_movies': 13, 'avg_per_movie': 61415039.0}
{'rank': 541, 'name': 'Anthony Gonzalez', 'WW_box': 798008101.0, 'num_movies': 3, 'avg_per_movie': 266002700.0}
{'rank': 542, 'name': 'Patrick Dempsey', 'WW_box': 797778618.0, 'num_movies': 9, 'avg_per_movie': 88642069.0}
{'rank': 543,

In [33]:
conn.commit()
c.close()
conn.close() 

In [38]:
conn = mysql.connector.connect(
                host = config.host,
                user = config.user,
                passwd = config.password,
                database = 'movie_insights'
                )
c = conn.cursor()

db_name = "movie_insights"
try:
    c.execute("USE {}".format(db_name))
except mysql.connector.Error as err:
    print("Database {} does not exists.".format(db_name))
    if err.errno == errorcode.ER_BAD_DB_ERROR:
        #create_database(c, db_name)
        print("Database {} created successfully.".format(db_name))
        conn.database = db_name
    else:
        print(err)
        exit(1)

query = ("""Select * from top_grossing_stars
                join actor_info using(name)
                join genre using(movie_id)
                group by genre_id
                Limit 100;""")

c.execute(query)
temp = c.fetchall()

temp





KeyboardInterrupt: 

In [39]:
c.close()
conn.close() 